In [1]:
from datasets import load_dataset

from bysearch import Engine
from bysearch.pipelines import HuggingFacePipeline, ONNXPipeline
from bysearch.backends import LocalBackend, PineconeBackend, ChromaBackend 

In [2]:
# backend = LocalBackend(text_column_name='text', id_column_name='url')
# backend = PineconeBackend(text_column_name='text', id_column_name='url', api_key='8d7794bb-a586-4dd9-a0b4-8d9dfd0fdf1a', environment ='gcp-starter', index_name='by-embeddings')
backend = ChromaBackend(text_column_name='text', id_column_name='url', type='persistent', collection_name='by-embeddings')

In [3]:
# pipeline = HuggingFacePipeline('KoichiYasuoka/roberta-small-belarusian', 'KoichiYasuoka/roberta-small-belarusian', 'cuda', max_context_length=127)
pipeline = ONNXPipeline('KoichiYasuoka/roberta-small-belarusian', 'onnx\\model.onnx', max_context_length=127)

In [4]:
dataset = load_dataset('mc4', 'be', split='validation')
search = Engine(dataset=dataset, pipeline=pipeline, backend=backend)
rez = search.search('аповесць беларускага пісьменніка Уладзіміра Караткевіча', verbose=False)
rez

Parameter 'function'=<function Engine.load_dataset.<locals>.<lambda> at 0x0000022529250720> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/1712 [00:00<?, ? examples/s]

,score,url,text,timestamp
0,6.629228,http://ikobrin.ru/martinov-52.php,﻿ Новыя паступленні музея імя Суворава | А. М....,2017-08-18T16:34:45Z
1,6.785421,http://zviazda.by/be/news/20150508/1431035548-...,﻿ Стаўленне да вайны — характарыстыка духоўнас...,2020-08-04T17:20:46Z
2,6.891847,https://be.m.wikipedia.org/wiki/%D0%9C%D1%96%D...,Мікалай Якаўлевіч Нікіфароўскі — Вікіпедыя\nМі...,2020-08-03T21:55:48Z
3,7.098675,http://ww.w.kamunikat.org/katalohbht.html?pub_...,"Арлоў Уладзімір, Айчына, частка першая - Белар...",2019-11-12T09:06:47Z
4,7.118645,http://3belarus.by/be/news/belarus-3-ushanue-p...,"""Беларусь 3"" ушануе памяць Валерыя Чкалава | Б...",2019-02-24T06:42:12Z


In [5]:
search.delete(rez['url'].tolist())
rez = search.search('аповесць беларускага пісьменніка Уладзіміра Караткевіча', verbose=False)
rez

,score,url,text,timestamp
0,7.238077,http://mininform.gov.by/news/stuzhka-navin/pre...,Прэзентацыя кнігі «Хуш Килесез — калі ласка!: ...,2019-02-17T06:57:53Z
1,7.278406,https://lit-bel.org/news/Alyaksey-Beli-adznach...,Аляксей Белы адзначае юбілей\nГалоўная » Навін...,2020-03-29T15:38:08Z
2,7.476275,https://www.sb.by/articles/darogam-maks-ma-tan...,«Дарогамі Максіма Танка»\nОпубликовано: 18.09....,2018-03-23T16:48:12Z
3,7.612226,http://kb.brl.by/index.php/home?view=featured,"Творчая сустрэча «Родны край, я цябе апяваю»\n...",2019-12-12T14:06:21Z
4,7.784564,https://www.kvitki.by/rus/bileti/teatr/drama/p...,Прэм'ера ''Смешныя людзі'' - 12.10.2018 - Наци...,2018-10-18T09:00:55Z


In [6]:
new_dataset = load_dataset('mc4', 'be', split='validation')
search.upsert(dataset=new_dataset)
rez = search.search('аповесць беларускага пісьменніка Уладзіміра Караткевіча', verbose=False)
rez

Map:   0%|          | 0/1712 [00:00<?, ? examples/s]

,score,url,text,timestamp
0,6.629228,http://ikobrin.ru/martinov-52.php,﻿ Новыя паступленні музея імя Суворава | А. М....,2017-08-18T16:34:45Z
1,6.785421,http://zviazda.by/be/news/20150508/1431035548-...,﻿ Стаўленне да вайны — характарыстыка духоўнас...,2020-08-04T17:20:46Z
2,6.891847,https://be.m.wikipedia.org/wiki/%D0%9C%D1%96%D...,Мікалай Якаўлевіч Нікіфароўскі — Вікіпедыя\nМі...,2020-08-03T21:55:48Z
3,7.098675,http://ww.w.kamunikat.org/katalohbht.html?pub_...,"Арлоў Уладзімір, Айчына, частка першая - Белар...",2019-11-12T09:06:47Z
4,7.118645,http://3belarus.by/be/news/belarus-3-ushanue-p...,"""Беларусь 3"" ушануе памяць Валерыя Чкалава | Б...",2019-02-24T06:42:12Z
